In [3]:
import pandas as pd
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LinearRegression
import requests
from bs4 import BeautifulSoup
from collections import Counter
import os
import requests
import shutil
import matplotlib.pyplot as plt
import math
import requests
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import r2_score


In [1]:
# This takes a long time dont run this

In [4]:
df = pd.read_csv("train_final.csv")


df = pd.get_dummies(df, columns=["property_type","neighbourhood_cleansed","neighbourhood_group_cleansed"], dtype=bool)

df = df.drop("room_type", axis=1)
# Drop columns with only one value
df = df.loc[:, df.nunique() > 1]


ranking_numeric = []

# Loop over each feature (skip target)
for feature in df.select_dtypes(include='number').columns:
    
    # Fill NaNs with mean per column
    df_nona = df.fillna(df.mean(numeric_only=True))
    
    X = df_nona[[feature]].values
    y = df_nona["price" ].values
    

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)

    ranking_numeric.append((feature, r2))



ranking_bool = []

for feature in df.select_dtypes(include='bool').columns:

    
    y_mean = np.ones(len(df["price" ].values))*df["price" ].values.mean()

    df_bool_true = df[df[feature]==True]
    df_bool_false = df[df[feature]==False]

    if(len(df_bool_true)==0 or len(df_bool_false)==0):
        continue
    

    
    y_bool_true_mean = np.ones(len(df["price" ].values))*df_bool_true["price" ].values.mean()
    y_bool_false_mean = np.ones(len(df["price" ].values))*df_bool_false["price" ].values.mean()

    if np.isnan(y_mean).any() or np.isnan(y_bool_true_mean).any():
        print(feature)
    
    mse_true = mean_squared_error(y_mean, y_bool_true_mean)
    mse_false = mean_squared_error(y_mean, y_bool_false_mean)

    ranking_bool.append((feature, max(mse_true,mse_false)))


# Sort by R^2 descending
ranking_numeric_sorted = sorted(ranking_numeric, key=lambda x: x[1], reverse=True)
# Sort
ranking_bool_sorted = sorted(ranking_bool, key=lambda x: x[1], reverse=True)

# Remove R^2
ranking_numeric_sorted = [r[0] for r in ranking_numeric_sorted]
ranking_bool_sorted = [r[0] for r in ranking_bool_sorted]

# Remove Price
ranking_numeric_sorted = ranking_numeric_sorted[1:]
# now merge lists

# Add first five numeric columns
ranking_features = ranking_numeric_sorted[0:5]


# Rank features
# Assuming all three are lists
features_ranked = []
features_ranked.extend(ranking_numeric_sorted[0:5])
ranking_numeric_sorted = ranking_numeric_sorted[5:]
features_ranked.extend(ranking_bool_sorted[0:10])
ranking_bool_sorted = ranking_bool_sorted[10:]

while ranking_numeric_sorted or ranking_bool_sorted:
    if ranking_numeric_sorted:
        features_ranked.append(ranking_numeric_sorted.pop(0))
    if ranking_bool_sorted:
        features_ranked.append(ranking_bool_sorted.pop(0))
    if ranking_bool_sorted:
        features_ranked.append(ranking_bool_sorted.pop(0))
    if ranking_bool_sorted:
        features_ranked.append(ranking_bool_sorted.pop(0))

features_ranked[0:30]

['accommodates',
 'bedrooms',
 'beds',
 'bathrooms',
 'comments_tfidf_group',
 'sound_bang_olufsen',
 'outdoor_pool_private',
 'oven_beko',
 'neighbourhood_cleansed_West 5',
 'stove_whirlpool',
 'exercise_equipment_workout_bench',
 'pool_view',
 'neighbourhood_cleansed_Haselhorst',
 'property_type_Boat',
 'property_type_Houseboat',
 'comments_tfidf_large group',
 'property_type_Entire townhouse',
 'neighbourhood_cleansed_Hellersdorf-Süd',
 'comments_tfidf_big group',
 'parking_free_carport',
 'property_type_Shared room in rental unit',
 'sqm',
 'property_type_Shared room in hostel',
 'property_type_Shared room in hotel',
 'comments_tfidf_family',
 'stove_gaggenau',
 'property_type_Private room in vacation home',
 'comments_tfidf_room',
 'property_type_Shared room in bed and breakfast',
 'property_type_Private room in cave']